In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Haar features for the facial features detection


In [0]:
# !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml -P drive/My\ Drive
# !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye_tree_eyeglasses.xml -P drive/My\ Drive
# !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml -P drive/My\ Drive
# !wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml -P drive/My\ Drive  
# !wget https://raw.githubusercontent.com/opencv/opencv_contrib/master/modules/face/data/cascades/haarcascade_mcs_nose.xml -P drive/My\ Drive

Importing library

In [0]:
import os
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split

initialising the cascade haar features

In [0]:
cascades_path = 'drive/My Drive/'
face_cascade = cv2.CascadeClassifier(cascades_path + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cascades_path + 'haarcascade_eye_tree_eyeglasses.xml')
nose_cascade = cv2.CascadeClassifier(cascades_path + 'haarcascade_mcs_nose.xml')

Face detection function. This one creates the oval shape around the face so that the unnecessary background doesn't come into picture

In [0]:
def face_detect(img):
  gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  if (len(faces) == 0 ):
    return None, None
  else:
    for (x , y, w, h) in faces:
      roi_gray = gray[y:y+h, x:x+w]
      roi_color = img[y:y+h, x:x+w]
      mask = np.zeros_like(roi_color)
      img_mask = np.zeros_like(img)
      rows, cols,_ = mask.shape
      img_mask = cv2.ellipse(img_mask,(int(rows/2)+x, int(cols/2)+y), (int((2*x+w)/3),int((2*y+h)/3.2)), angle=0, startAngle=0, endAngle=360, color=(255,255,255), thickness=-1)    
      img = np.bitwise_and(img_mask, img)
      img = img[y:y+h, x:x+w]
      img_gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
  return img,img_gray

Reading the dataset images one by one labelling them accordingly. Image is resized to (64, 64) dimension due to memory contraints.

In [0]:
images = []
label = []
filenames = os.listdir("/content/drive/My Drive/AVR_data")
for filename in filenames:
  sub_filenames = os.listdir("/content/drive/My Drive/AVR_data/"+filename)
  for sub_file in sub_filenames:
    image = cv2.imread("/content/drive/My Drive/AVR_data/"+filename + "/" +sub_file)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image, image_gray = face_detect(image)
    if image is not None:
      image_g = cv2.resize(image_gray , (64, 64))
      images.append(image_g)
      label.append(filename)

Completing the labelling process

In [0]:
labels = np.array(label)
label_encoder = preprocessing.LabelEncoder()
enc_labels = label_encoder.fit_transform(labels) 
labels_unique = label_encoder.classes_
labels_unique = np.array(labels_unique)
enc_labels = np.array(enc_labels)

In [0]:
images = np.array(images)

Flatten the single image from (64, 64) to (4096, 1) and then creating a set of those array by stacking them column wise

In [0]:
def flatten_array(images):
  X_data = []
  for image in images:
      X_data.append(image.flatten())
  X_data = np.array(X_data)
  return X_data.T

euclideanDistance function calculates the distance between two vectors  

In [0]:
def euclideanDistance(vector1, vector2):
  return np.linalg.norm(vector1 - vector2)

leastEucDisN first invokes the distance calculation then sorts those distance in ascending order and returns the top N nearest neighbours labels

In [0]:
def leastEucDistN(matrix, vector, N):
    distance_list = []
    for i in range(((matrix.T).shape)[0]):
        euc_dist = euclideanDistance(vector, (matrix.T)[i]) 
        distance_list.append([euc_dist, i])
    distance_list = sorted(distance_list)
    min_ind_list = []
    for i in range(N):
        min_ind_list.append(distance_list[i][1])
    return min_ind_list

The sorted labels are then compared with the test label if they are in top N distance

In [0]:
def PCA_sklearn_predictions(X_train_pca ,y_train, X_test_pca, N):
  dist = []
  predictions = []
  for test_col in X_test_pca:
    dist = leastEucDistN(X_train_pca.T, test_col , N)
    l2 = []
    for i in range(len(dist)):
      l2.append(y_train[dist[i]])
    predictions.append(l2)
  return predictions

This function then computes the count of correctly labelled predictions and then returns the accuracy score

In [0]:
def PCA_accuracyN(predictions, y_test, N):
    count = 0
    for i in range(len(y_test)):
      if ((y_test[i]) in predictions[i]):
        count += 1
    print("PCA top:", N, "Accuracy: ", (count/(len(y_test)-1))*100)

Predictions

Image dataset is flattened and stacked column wise. the total images read properly are 624. Some image enhachments are required to read the whole 824 images properly

In [14]:
image = flatten_array(images)
image.shape

(4096, 624)

Splittng the test train data set

In [0]:
(X_train, X_test, y_train, y_test) = train_test_split(image.T, enc_labels, test_size = 0.25, random_state = 42)

The test train splits the dataset into 468 and 156 images respectively

In [16]:
print(X_train.shape)
print(X_test.shape)

(468, 4096)
(156, 4096)


Next task is to reduce the dimensions of the images. Since a single image is having the 4096 features, the distance matric will take a lot of time to evaluate and also will include some un necessary features which are not present in all the images. One method is to use **PCA** another method that could be used is **LDA**. Here LDA is used.

In [0]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

After some hit and trial in the number of features. 20 is the best number of features that describes the dataset

In [0]:
lda = LDA()

# apply PCA transformation
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

The features are reduced whitout disturbing the image array

In [47]:
print(X_test_lda.shape)
print(X_train_lda.shape)

(156, 40)
(468, 40)


These are top N accuracies of the dataset

In [67]:
predictions = PCA_sklearn_predictions(X_train_lda, y_train, X_test_lda , 1)
PCA_accuracyN(predictions, y_test, 1)
predictions = PCA_sklearn_predictions(X_train_lda, y_train, X_test_lda , 3)
PCA_accuracyN(predictions, y_test, 3)
predictions = PCA_sklearn_predictions(X_train_lda, y_train, X_test_lda , 5)
PCA_accuracyN(predictions, y_test, 5)
predictions = PCA_sklearn_predictions(X_train_lda, y_train, X_test_lda , 10)
PCA_accuracyN(predictions, y_test, 10)

PCA top: 1 Accuracy:  60.64516129032258
PCA top: 3 Accuracy:  74.83870967741936
PCA top: 5 Accuracy:  80.64516129032258
PCA top: 10 Accuracy:  87.09677419354838
